In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [4]:
tx_path = pd.read_csv(pd_data_base_path + 'Transaction Path.csv')
tx_detail = pd.read_csv(pd_data_base_path + 'Transaction Detail.csv')
acc_info = pd.read_csv(pd_data_base_path + 'Account Information.csv')
acc_holders = pd.read_csv(pd_data_base_path + 'Account Holders.csv')

In [5]:
tx_path.head()

,Transaction ID,Account_To,Account_From
0,1957155,27356852,76206810
1,2147025,44242297,24826358
2,3065073,10295384,52104303
3,6622100,45519330,69315008
4,14877473,28680375,44586370


In [6]:
tx_path.rename(columns={'Account_To':'Account To','Account_From':'Account From'}, inplace=True)

In [7]:
acc_info.shape

(3000, 5)

In [8]:
acc_info[acc_info['Account Number'].notnull()]

,Account Number,Account Type,Account Holder ID,Balance Date,Balance
0,10005367,Platinum,70390615,2023-01-31,728.25
1,10011977,Basic,20123998,2023-01-31,676.54
2,10024680,Platinum,54374080,2023-01-31,567.46
3,10031238,Basic,97027297,2023-01-31,576.52
4,10034341,Joint,"89920386, 97325900",2023-01-31,390.39
...,...,...,...,...,...
2995,99734848,Gold,56623581,2023-01-31,354.29
2996,99760030,Gold,66659633,2023-01-31,988.00
2997,99791709,Gold,94872412,2023-01-31,15.05
2998,99877007,Basic,90069774,2023-01-31,582.00


In [9]:
acc_info['Account Holder ID'] = acc_info['Account Holder ID'].apply(lambda x: x.split(', '))

In [10]:
acc_info = acc_info.explode('Account Holder ID')

In [11]:
acc_info['Account Holder ID'] = acc_info['Account Holder ID'].astype('int')
acc_info.head(10)

,Account Number,Account Type,Account Holder ID,Balance Date,Balance
0,10005367,Platinum,70390615,2023-01-31,728.25
1,10011977,Basic,20123998,2023-01-31,676.54
2,10024680,Platinum,54374080,2023-01-31,567.46
3,10031238,Basic,97027297,2023-01-31,576.52
4,10034341,Joint,89920386,2023-01-31,390.39
4,10034341,Joint,97325900,2023-01-31,390.39
5,10079965,Basic,89460366,2023-01-31,799.27
6,10091076,Platinum,53258828,2023-01-31,831.83
7,10231977,Basic,41402857,2023-01-31,61.63
8,10232603,Gold,8882143,2023-01-31,732.55


In [12]:
acc_holders[(acc_holders['Contact Number']/1000000000>=8) | (acc_holders['Contact Number']/1000000000<=6)]

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address


In [13]:
acc_holders['Contact Number'] = acc_holders['Contact Number'].astype('str')
acc_holders['Contact Number'] = '0' + acc_holders['Contact Number']

In [14]:
acc_holders.head()

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park
1,20123998,Maynord Surgeoner,21/08/1997,07716107305,6422 Buena Vista Plaza
2,54374080,Giraldo Kimbley,22/03/1995,07489940612,93005 Summer Ridge Avenue
3,97027297,Blake Dudson,30/06/1955,07253587445,2 Huxley Hill
4,89920386,Ajay Douce,19/12/1930,07395580534,90176 Miller Alley


In [15]:
df = pd.merge(tx_path, tx_detail, on='Transaction ID', how='left')
df = pd.merge(df, acc_info, left_on='Account From', right_on='Account Number', how='left')
df = pd.merge(df, acc_holders, on='Account Holder ID', how='left')

In [16]:
df.head()

,Transaction ID,Account To,Account From,Transaction Date,Value,Cancelled?,Account Number,Account Type,Account Holder ID,Balance Date,Balance,Name,Date of Birth,Contact Number,First Line of Address
0,1957155,27356852,76206810,2023-02-01,128.78,N,76206810,Platinum,91369374,2023-01-31,887.63,Karolina Lorraine,06/06/1955,07353176639,5 Helena Junction
1,2147025,44242297,24826358,2023-02-09,170.19,N,24826358,Gold,12728922,2023-01-31,923.46,Brinna Bootherstone,04/11/1974,07340315466,1657 Bartillon Park
2,3065073,10295384,52104303,2023-02-06,87.57,N,52104303,Gold,2746224,2023-01-31,810.97,Fulton Cush,17/11/1935,07818008990,2484 Hagan Pass
3,6622100,45519330,69315008,2023-02-07,85.76,N,69315008,Gold,78932422,2023-01-31,923.08,Ferdy Jehaes,19/08/1953,07802788844,70 Carberry Junction
4,14877473,28680375,44586370,2023-02-02,84.65,N,44586370,Gold,52060262,2023-01-31,440.56,Barbee Chatband,16/10/1978,07796180629,9807 Lakewood Gardens Way


In [17]:
df = df[df['Cancelled?']!='Y']
df = df[df['Value']>1000]
df = df[df['Account Type']!='Platinum']
df = df[['Transaction ID', 'Account To', 'Transaction Date', 'Value', 'Account Number', 'Account Type', 'Balance Date', 'Balance', 'Name', 'Date of Birth', 'Contact Number', 'First Line of Address']]

In [18]:
df.head()

,Transaction ID,Account To,Transaction Date,Value,Account Number,Account Type,Balance Date,Balance,Name,Date of Birth,Contact Number,First Line of Address
25,46779876,21694967,2023-02-02,1019.5,82023979,Gold,2023-01-31,744.37,Barb Eminson,20/07/1974,07748168711,3149 American Ash Hill
37,57491925,19094015,2023-02-12,1761.1,65589565,Gold,2023-01-31,321.99,Annabel Medd,17/02/1959,07175713923,24 Warner Junction
86,108536548,86207903,2023-02-06,1085.4,10553619,Basic,2023-01-31,756.20,Dud Gaunt,04/12/1983,07681115087,3 Canary Avenue
136,176916326,33632099,2023-02-12,1955.6,18185362,Basic,2023-01-31,609.36,Agace Sporle,28/02/1999,07182359874,96 Emmet Pass
141,180899074,89503991,2023-02-12,1030.7,91605034,Basic,2023-01-31,560.96,Guthrie Denidge,20/01/1970,07398074919,773 South Terrace


In [19]:
df.shape

(476, 12)